<a href="https://colab.research.google.com/github/Danc2050/Distributed-Computing-Scripts/blob/master/google-colab/GoogleColabCPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title <strong> <h1>📔 Colab CPU Notebook</strong></h1> #{ output-height: 8, form-width: "100%", display-mode: "form" }

#@markdown <h4>↖️ Click the ▶ button after deciding on the options below.</h4>
#@markdown <h4>📜 See <a href="https://github.com/Danc2050/Distributed-Computing-Scripts/tree/master/google-colab" target="_blank">source code</a> for more information.</h4>
#@markdown <h4>💡 Keep each notebook <strong>open</strong> in your browser to prevent disconnection. (📌 pin the tab or 🗗 move it to a dedicated window)</h4>
#@markdown <h4>🤷‍♂️ Optionally, create a GIMPS/PrimeNet account <a href="https://www.mersenne.org/update/" target="_blank">here</a>.</h4>

prime_ID = 'Default' #@param ['Default'] {allow-input: true}
computer_name = 'Default' #@param ['Default'] {allow-input: true}
type_of_work = '100 - First time LL tests' #@param ['0 - Whatever makes the most sense', '100 - First time LL tests', '101 - Double-check LL tests', '102 - World record LL tests', '104 - 100 million digit LL tests', '150 - First time PRP tests', '151 - Double-check PRP tests', '152 - World record PRP tests', '154 - 100 million digit PRP tests']

# Load & cd into gdrive for persistent data
from google.colab import drive
import os
drive.mount('/content/gdrive')
%cd gdrive/'My Drive'

def run():
  # Run Mprime
  !cd mprime_cpu && chmod 777 mprime && ./mprime -d # `-d` prevents VM from shutting down.

def install():
  # Download/Install/Configure Prime95
  !rm mprime.sh # remove any old download of the Prime95 script
  !wget https://raw.github.com/tdulcet/Linux-System-Information/master/info.sh -qO - | bash -s # Check System Info
  !wget https://raw.github.com/tdulcet/Distributed-Computing-Scripts/master/mprime.sh -q # Download Prime95 script
  !sed -i '/^nohup / s/^/# /' mprime.sh # Do not start Prime95
  !sed -i '/^crontab / s/^/# /' mprime.sh # Do not create a cronjob
  !sed -i 's/"mprime"/"mprime_cpu"/' mprime.sh # Name the folder specific to the runtime type
  !bash -- mprime.sh \""$prime_ID\"" \""$computer_name\"" $type_of_work # Run script

# use/cleanup input from user
prime_ID = 'psu' if prime_ID.lower() == 'default' else prime_ID
computers_name = "" if computer_name.lower() == 'default' else computer_name
type_of_work = type_of_work.split("-")[0].rstrip()

if not os.path.exists('mprime_cpu'):
  install()
run()

print("Gracefully exiting...")